In [ ]:
from langchain.chat_models import ChatOllama

chat = ChatOllama(model="mistral:latest")

In [ ]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian."),
  AIMessage(content="Ciao, mi chiamo Paolo"),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also what is your name?"),
]

chat.predict_messages(messages)